In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
dcn_model = tf.keras.models.load_model('models/best_dcn.h5')

In [3]:
interaction_df = pd.read_csv('metadata/interactions.csv').drop('Unnamed: 0',axis=1)
df_segments = pd.read_csv('metadata/zip-speciality-segments.csv').drop('Unnamed: 0',axis=1)


In [4]:
user_id_map = {id: idx for idx, id in enumerate(interaction_df['PERSON_ID'].unique())}
provider_id_map = {id: idx for idx, id in enumerate(interaction_df['ProviderCode'].unique())}


In [5]:
df_segments.head(2)

,zip_code,FACILITYCODE,ProviderCode,SpecialtyCode,SpecialtyDescription
0,33328,100360,YVMCR,PS628,Orthopedic Spine Surgery
1,20900,6E1D9D,YVMCR,PS628,Orthopedic Spine Surgery


In [6]:
interaction_df.head(2)

,PERSON_ID,ProviderCode,SurveyAnswerScore,RecommendStar,ratings,age,survey_score,star_rating,zip_code,FACILITYCODE,SpecialtyCode,SpecialtyDescription,GENDER_Female,GENDER_Male
0,00123db9a36e9f9277d6ce32c3fdd1e49809845399741b...,XF6BG,2.500000,5.0,3.75,39.0,3.05,3.67,34950,81DB7D,PS305,Family Medicine,1,0
1,32e8750a2e31a34c71d700df32a773adcab8db3109117b...,XF6BG,4.708333,5.0,4.85,39.0,3.05,3.67,34950,81DB7D,PS305,Family Medicine,0,1


In [7]:
def get_recommendations(user_id, zip_code, specialty_code, top_n=5):
    # Filter providers based on zip code and specialty code
    filtered_providers = df_segments[(df_segments['zip_code'] == zip_code) &
                                     (df_segments['SpecialtyCode'] == specialty_code)]

    filtrd_prov_ids = filtered_providers['ProviderCode'].unique().tolist()

    filtered_providers = interaction_df[interaction_df['ProviderCode'].isin(filtrd_prov_ids)]
    filtered_providers = filtered_providers[['ProviderCode','survey_score','star_rating']]  
    filtered_providers = filtered_providers.drop_duplicates()
     
    
   
    if filtered_providers.empty:
        return []

    # Prepare user features
    user_features = interaction_df[interaction_df['PERSON_ID'] == user_id]
    
    if user_features.empty:
        return []

    user_idx = user_id_map[user_id]
    user_age = user_features['age'].values[0]
    user_gender_male = user_features['GENDER_Male'].values[0]
    user_gender_female = user_features['GENDER_Female'].values[0]

    # Prepare inputs for the model
    user_input = np.array([user_idx]).reshape(-1, 1)
    user_age_input = np.array([user_age]).reshape(-1, 1)
    user_gender_input = np.array([[user_gender_male, user_gender_female]])

    recommendations = []

    # Predict ratings for each filtered provider
    for _, provider in filtered_providers.iterrows():
        provider_idx = provider_id_map[provider['ProviderCode']]
        provider_survey_score = provider['survey_score']
        provider_star_rating = provider['star_rating']

        provider_input = np.array([provider_idx]).reshape(-1, 1)
        provider_survey_score_input = np.array([provider_survey_score]).reshape(-1, 1)
        provider_star_rating_input = np.array([provider_star_rating]).reshape(-1, 1)

        # Predict rating
        predicted_rating = dcn_model.predict([
            user_input,
            user_age_input,
            user_gender_input,
            provider_input,
            provider_survey_score_input,
            provider_star_rating_input
        ])

        recommendations.append((provider['ProviderCode'], predicted_rating[0][0]))

    # Sort providers based on predicted ratings
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)

    # Return top N providers
    return recommendations[:top_n]


In [8]:
user_id = "3a0c21ce669eb780b459e7ba4f7955d5965a561cd6d1be9fddcaa8e9e918b3fb"
zip_code = 20900
specialty_code = "PS628"

In [9]:
# Get recommendations
top_providers = get_recommendations(user_id, zip_code, specialty_code, top_n=5)
print("Top Providers:", top_providers)

1/1 [==============================] - 0s 23ms/step
Top Providers: [('XYN6P6J', 16.17348), ('XF8K7', 15.852696), ('YVMCR', 15.729217), ('YFGQD', 15.622549)]


In [11]:
top_providers

[('XYN6P6J', 16.17348),
 ('XF8K7', 15.852696),
 ('YVMCR', 15.729217),
 ('YFGQD', 15.622549)]